# Sentiment Analysis Visualization

Visualize sentiment analysis results from German political tweets.

**Use cases:** Analyzing sentiment towards parties, politicians, and policy topics.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
from sqlalchemy import text
import yaml
import sys

# Database connection
sys.path.insert(0, str(Path.cwd().parent / 'src'))
from xminer.io.db import engine

In [ ]:
# --- Load parameters from config ---
PARAMS_FILE = Path("../src/xminer/config/parameters.yml")
assert PARAMS_FILE.exists(), f"parameters.yml not found: {PARAMS_FILE}"

with PARAMS_FILE.open("r", encoding="utf-8") as f:
    params = yaml.safe_load(f) or {}

YEAR = int(params.get("year", 2026))
MONTH = int(params.get("month", 1))
YM = f"{YEAR:04d}{MONTH:02d}"

STAND_TEXT_DE = f"Erhoben für {MONTH:02d}/{YEAR}"
STAND_TEXT_EN = f"Collected for {MONTH:02d}/{YEAR}"

# Output directory for graphics
GRAPHICS_BASE_DIR = Path(params.get("graphics_base_dir", "../outputs"))
GRAPHICS_DIR = GRAPHICS_BASE_DIR / YM / "graphics" / "sentiment"
GRAPHICS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Graphics will be saved to: {GRAPHICS_DIR}")
print(f"Period: {YEAR}-{MONTH:02d}")

In [ ]:
# Party colors (consistent with other notebooks)
PARTY_COLORS = {
    "CDU/CSU": "#000000",
    "CDU": "#000000",
    "CSU": "#000000",
    "SPD": "#E3000F",
    "GRÜNE": "#1AA64A",
    "Grüne": "#1AA64A",
    "BÜNDNIS 90/DIE GRÜNEN": "#1AA64A",
    "DIE LINKE.": "#BE3075",
    "LINKE": "#BE3075",
    "Linke": "#BE3075",
    "FDP": "#FFED00",
    "AFD": "#009EE0",
    "AfD": "#009EE0",
    "BSW": "#009688",
}

# Sentiment colors
SENTIMENT_COLORS = {
    "positive": "#2ecc71",
    "negative": "#e74c3c",
    "neutral": "#95a5a6",
    "mixed": "#f39c12"
}

def get_party_color(party: str) -> str:
    """Get color for a party."""
    return PARTY_COLORS.get(party, "#888888")

def save_fig(fig, save_name: str, width: int = 1200, height: int = 675, scale: int = 2):
    """Save figure to GRAPHICS_DIR (optimized for social media)."""
    save_path = GRAPHICS_DIR / f"{save_name}.png"
    fig.write_image(save_path, width=width, height=height, scale=scale)
    print(f"✅ Saved: {save_path}")

## 1. Load Sentiment Data from Database

In [ ]:
# Load sentiment analysis results
with engine.connect() as conn:
    df_sentiment = pd.read_sql(text("""
        SELECT 
            ts.tweet_id,
            ts.author_id,
            ts.topic,
            ts.topic_type,
            ts.sentiment,
            ts.score,
            ts.confidence,
            ts.reasoning,
            ts.analyzed_at,
            t.username,
            t.text as tweet_text,
            t.created_at
        FROM tweet_sentiments ts
        JOIN tweets t ON ts.tweet_id = t.tweet_id
        ORDER BY ts.analyzed_at DESC
    """), conn)

print(f"Total sentiment records: {len(df_sentiment):,}")
print(f"Unique tweets analyzed: {df_sentiment['tweet_id'].nunique():,}")
print(f"Unique topics: {df_sentiment['topic'].nunique()}")
df_sentiment.head()

## 2. Sentiment Distribution by Topic Type

In [ ]:
# Sentiment distribution by topic type
df_topic_type = df_sentiment.groupby(['topic_type', 'sentiment']).size().reset_index(name='count')

# German version
fig_de = px.bar(
    df_topic_type,
    x='topic_type',
    y='count',
    color='sentiment',
    color_discrete_map=SENTIMENT_COLORS,
    barmode='group',
    labels={'topic_type': 'Thementyp', 'count': 'Anzahl', 'sentiment': 'Sentiment'}
)
fig_de.update_layout(
    title=dict(
        text=f"Sentiment-Verteilung nach Thementyp<br><sub style='font-size:0.85em;'>{STAND_TEXT_DE}</sub>",
        x=0.5, xanchor='center', font=dict(size=22)
    ),
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#1a1a1a',
    font=dict(color='white', size=14),
    height=500,
    margin=dict(t=100, b=60, l=60, r=60),
    xaxis=dict(gridcolor='#333333'),
    yaxis=dict(gridcolor='#333333')
)
save_fig(fig_de, "sentiment_by_topic_type_de")
fig_de.show()

# English version
fig_en = px.bar(
    df_topic_type,
    x='topic_type',
    y='count',
    color='sentiment',
    color_discrete_map=SENTIMENT_COLORS,
    barmode='group',
    labels={'topic_type': 'Topic Type', 'count': 'Count', 'sentiment': 'Sentiment'}
)
fig_en.update_layout(
    title=dict(
        text=f"Sentiment Distribution by Topic Type<br><sub style='font-size:0.85em;'>{STAND_TEXT_EN}</sub>",
        x=0.5, xanchor='center', font=dict(size=22)
    ),
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#1a1a1a',
    font=dict(color='white', size=14),
    height=500,
    margin=dict(t=100, b=60, l=60, r=60),
    xaxis=dict(gridcolor='#333333'),
    yaxis=dict(gridcolor='#333333')
)
save_fig(fig_en, "sentiment_by_topic_type_en")
fig_en.show()

## 3. Average Sentiment Score by Party

In [ ]:
# Filter for party topics only
df_parties = df_sentiment[df_sentiment['topic_type'] == 'party'].copy()

if len(df_parties) > 0:
    # Calculate average sentiment score per party
    df_party_avg = df_parties.groupby('topic').agg({
        'score': 'mean',
        'tweet_id': 'count',
        'confidence': 'mean'
    }).reset_index()
    df_party_avg.columns = ['party', 'avg_score', 'mention_count', 'avg_confidence']
    df_party_avg = df_party_avg.sort_values('avg_score', ascending=True)

    # Get colors for each party
    colors = [get_party_color(p) for p in df_party_avg['party']]

    # German version
    fig_de = go.Figure(go.Bar(
        x=df_party_avg['avg_score'],
        y=df_party_avg['party'],
        orientation='h',
        marker_color=colors,
        text=[f"{s:.2f}" for s in df_party_avg['avg_score']],
        textposition='outside',
        textfont=dict(color='white', size=14),
        customdata=df_party_avg[['mention_count', 'avg_confidence']].values,
        hovertemplate=(
            "<b>%{y}</b><br>"
            "Durchschn. Score: %{x:.2f}<br>"
            "Erwähnungen: %{customdata[0]}<br>"
            "Konfidenz: %{customdata[1]:.2f}<extra></extra>"
        )
    ))
    fig_de.add_vline(x=0, line_dash="dash", line_color="gray")
    fig_de.update_layout(
        title=dict(
            text=f"Durchschnittlicher Sentiment-Score nach Partei<br><sub style='font-size:0.85em;'>{STAND_TEXT_DE}</sub>",
            x=0.5, xanchor='center', font=dict(size=22)
        ),
        xaxis_title='Sentiment Score (-1 bis +1)',
        yaxis_title='',
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14),
        height=max(400, 60 * len(df_party_avg)),
        xaxis=dict(range=[-1.1, 1.1], gridcolor='#333333'),
        yaxis=dict(gridcolor='#333333', tickfont=dict(size=16)),
        margin=dict(t=100, b=60, l=120, r=100)
    )
    save_fig(fig_de, "sentiment_score_by_party_de")
    fig_de.show()

    # English version
    fig_en = go.Figure(go.Bar(
        x=df_party_avg['avg_score'],
        y=df_party_avg['party'],
        orientation='h',
        marker_color=colors,
        text=[f"{s:.2f}" for s in df_party_avg['avg_score']],
        textposition='outside',
        textfont=dict(color='white', size=14),
        customdata=df_party_avg[['mention_count', 'avg_confidence']].values,
        hovertemplate=(
            "<b>%{y}</b><br>"
            "Avg Score: %{x:.2f}<br>"
            "Mentions: %{customdata[0]}<br>"
            "Confidence: %{customdata[1]:.2f}<extra></extra>"
        )
    ))
    fig_en.add_vline(x=0, line_dash="dash", line_color="gray")
    fig_en.update_layout(
        title=dict(
            text=f"Average Sentiment Score by Party<br><sub style='font-size:0.85em;'>{STAND_TEXT_EN}</sub>",
            x=0.5, xanchor='center', font=dict(size=22)
        ),
        xaxis_title='Sentiment Score (-1 to +1)',
        yaxis_title='',
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14),
        height=max(400, 60 * len(df_party_avg)),
        xaxis=dict(range=[-1.1, 1.1], gridcolor='#333333'),
        yaxis=dict(gridcolor='#333333', tickfont=dict(size=16)),
        margin=dict(t=100, b=60, l=120, r=100)
    )
    save_fig(fig_en, "sentiment_score_by_party_en")
    fig_en.show()
else:
    print("No party sentiment data available")

## 4. Sentiment Distribution for Policy Topics

In [ ]:
# Filter for policy topics
df_policy = df_sentiment[df_sentiment['topic_type'] == 'policy'].copy()

if len(df_policy) > 0:
    df_policy_sentiment = df_policy.groupby(['topic', 'sentiment']).size().unstack(fill_value=0)
    df_policy_sentiment['total'] = df_policy_sentiment.sum(axis=1)
    df_policy_sentiment = df_policy_sentiment.sort_values('total', ascending=True)

    # German version
    fig_de = go.Figure()
    for sentiment in ['positive', 'negative', 'neutral', 'mixed']:
        if sentiment in df_policy_sentiment.columns:
            fig_de.add_trace(go.Bar(
                name=sentiment.capitalize(),
                y=df_policy_sentiment.index,
                x=df_policy_sentiment[sentiment],
                orientation='h',
                marker_color=SENTIMENT_COLORS[sentiment]
            ))
    fig_de.update_layout(
        barmode='stack',
        title=dict(
            text=f"Sentiment-Verteilung nach Politikbereich<br><sub style='font-size:0.85em;'>{STAND_TEXT_DE}</sub>",
            x=0.5, xanchor='center', font=dict(size=22)
        ),
        xaxis_title='Anzahl Erwähnungen',
        yaxis_title='',
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14),
        height=max(500, 50 * len(df_policy_sentiment)),
        legend_title='Sentiment',
        xaxis=dict(gridcolor='#333333'),
        yaxis=dict(gridcolor='#333333', tickfont=dict(size=14)),
        margin=dict(t=100, b=60, l=150, r=60)
    )
    save_fig(fig_de, "sentiment_by_policy_de")
    fig_de.show()

    # English version
    fig_en = go.Figure()
    for sentiment in ['positive', 'negative', 'neutral', 'mixed']:
        if sentiment in df_policy_sentiment.columns:
            fig_en.add_trace(go.Bar(
                name=sentiment.capitalize(),
                y=df_policy_sentiment.index,
                x=df_policy_sentiment[sentiment],
                orientation='h',
                marker_color=SENTIMENT_COLORS[sentiment]
            ))
    fig_en.update_layout(
        barmode='stack',
        title=dict(
            text=f"Sentiment Distribution by Policy Area<br><sub style='font-size:0.85em;'>{STAND_TEXT_EN}</sub>",
            x=0.5, xanchor='center', font=dict(size=22)
        ),
        xaxis_title='Number of Mentions',
        yaxis_title='',
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14),
        height=max(500, 50 * len(df_policy_sentiment)),
        legend_title='Sentiment',
        xaxis=dict(gridcolor='#333333'),
        yaxis=dict(gridcolor='#333333', tickfont=dict(size=14)),
        margin=dict(t=100, b=60, l=150, r=60)
    )
    save_fig(fig_en, "sentiment_by_policy_en")
    fig_en.show()
else:
    print("No policy sentiment data available")

## 5. Sentiment Score by Politician

In [ ]:
# Filter for politician topics
df_politicians = df_sentiment[df_sentiment['topic_type'] == 'politician'].copy()

if len(df_politicians) > 0:
    df_pol_avg = df_politicians.groupby('topic').agg({
        'score': 'mean',
        'tweet_id': 'count',
        'confidence': 'mean'
    }).reset_index()
    df_pol_avg.columns = ['politician', 'avg_score', 'mention_count', 'avg_confidence']
    df_pol_avg = df_pol_avg.sort_values('avg_score', ascending=True)

    # German version
    fig_de = go.Figure(go.Bar(
        x=df_pol_avg['avg_score'],
        y=df_pol_avg['politician'],
        orientation='h',
        marker=dict(color=df_pol_avg['avg_score'], colorscale='RdYlGn', cmin=-1, cmax=1, colorbar=dict(title='Score')),
        text=[f"{s:.2f}" for s in df_pol_avg['avg_score']],
        textposition='outside',
        textfont=dict(color='white', size=14),
        customdata=df_pol_avg[['mention_count', 'avg_confidence']].values,
        hovertemplate="<b>%{y}</b><br>Score: %{x:.2f}<br>Erwähnungen: %{customdata[0]}<extra></extra>"
    ))
    fig_de.add_vline(x=0, line_dash="dash", line_color="gray")
    fig_de.update_layout(
        title=dict(
            text=f"Durchschnittlicher Sentiment-Score nach Politiker<br><sub style='font-size:0.85em;'>{STAND_TEXT_DE}</sub>",
            x=0.5, xanchor='center', font=dict(size=22)
        ),
        xaxis_title='Sentiment Score (-1 bis +1)',
        yaxis_title='',
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14),
        height=max(400, 50 * len(df_pol_avg)),
        xaxis=dict(range=[-1.1, 1.1], gridcolor='#333333'),
        yaxis=dict(gridcolor='#333333', tickfont=dict(size=14)),
        margin=dict(t=100, b=60, l=180, r=100)
    )
    save_fig(fig_de, "sentiment_score_by_politician_de")
    fig_de.show()

    # English version
    fig_en = go.Figure(go.Bar(
        x=df_pol_avg['avg_score'],
        y=df_pol_avg['politician'],
        orientation='h',
        marker=dict(color=df_pol_avg['avg_score'], colorscale='RdYlGn', cmin=-1, cmax=1, colorbar=dict(title='Score')),
        text=[f"{s:.2f}" for s in df_pol_avg['avg_score']],
        textposition='outside',
        textfont=dict(color='white', size=14),
        customdata=df_pol_avg[['mention_count', 'avg_confidence']].values,
        hovertemplate="<b>%{y}</b><br>Score: %{x:.2f}<br>Mentions: %{customdata[0]}<extra></extra>"
    ))
    fig_en.add_vline(x=0, line_dash="dash", line_color="gray")
    fig_en.update_layout(
        title=dict(
            text=f"Average Sentiment Score by Politician<br><sub style='font-size:0.85em;'>{STAND_TEXT_EN}</sub>",
            x=0.5, xanchor='center', font=dict(size=22)
        ),
        xaxis_title='Sentiment Score (-1 to +1)',
        yaxis_title='',
        plot_bgcolor='#1a1a1a',
        paper_bgcolor='#1a1a1a',
        font=dict(color='white', size=14),
        height=max(400, 50 * len(df_pol_avg)),
        xaxis=dict(range=[-1.1, 1.1], gridcolor='#333333'),
        yaxis=dict(gridcolor='#333333', tickfont=dict(size=14)),
        margin=dict(t=100, b=60, l=180, r=100)
    )
    save_fig(fig_en, "sentiment_score_by_politician_en")
    fig_en.show()
else:
    print("No politician sentiment data available")

## 6. Overall Sentiment Distribution (Pie Chart)

In [ ]:
# Overall sentiment distribution
sentiment_counts = df_sentiment['sentiment'].value_counts()

# German version
fig_de = go.Figure(go.Pie(
    labels=sentiment_counts.index,
    values=sentiment_counts.values,
    marker=dict(colors=[SENTIMENT_COLORS.get(s, '#888888') for s in sentiment_counts.index]),
    textinfo='label+percent',
    textfont=dict(size=16),
    hovertemplate="%{label}<br>Anzahl: %{value:,}<br>Anteil: %{percent}<extra></extra>"
))
fig_de.update_layout(
    title=dict(
        text=f"Gesamte Sentiment-Verteilung<br><sub style='font-size:0.85em;'>{STAND_TEXT_DE}</sub>",
        x=0.5, xanchor='center', font=dict(size=22)
    ),
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#1a1a1a',
    font=dict(color='white', size=14),
    height=600,
    margin=dict(t=100, b=40, l=40, r=40)
)
save_fig(fig_de, "sentiment_distribution_pie_de")
fig_de.show()

# English version
fig_en = go.Figure(go.Pie(
    labels=sentiment_counts.index,
    values=sentiment_counts.values,
    marker=dict(colors=[SENTIMENT_COLORS.get(s, '#888888') for s in sentiment_counts.index]),
    textinfo='label+percent',
    textfont=dict(size=16),
    hovertemplate="%{label}<br>Count: %{value:,}<br>Share: %{percent}<extra></extra>"
))
fig_en.update_layout(
    title=dict(
        text=f"Overall Sentiment Distribution<br><sub style='font-size:0.85em;'>{STAND_TEXT_EN}</sub>",
        x=0.5, xanchor='center', font=dict(size=22)
    ),
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#1a1a1a',
    font=dict(color='white', size=14),
    height=600,
    margin=dict(t=100, b=40, l=40, r=40)
)
save_fig(fig_en, "sentiment_distribution_pie_en")
fig_en.show()

## 7. Top Topics by Mention Count

In [ ]:
# Top 15 most mentioned topics
df_top_topics = df_sentiment.groupby(['topic', 'topic_type']).agg({'score': 'mean', 'tweet_id': 'count'}).reset_index()
df_top_topics.columns = ['topic', 'topic_type', 'avg_score', 'mentions']
df_top_topics = df_top_topics.nlargest(15, 'mentions').sort_values('mentions', ascending=True)

# German version
fig_de = go.Figure(go.Bar(
    x=df_top_topics['mentions'],
    y=df_top_topics['topic'],
    orientation='h',
    marker=dict(color=df_top_topics['avg_score'], colorscale='RdYlGn', cmin=-1, cmax=1, colorbar=dict(title='Score')),
    text=[f"{m:,} ({s:.2f})" for m, s in zip(df_top_topics['mentions'], df_top_topics['avg_score'])],
    textposition='outside',
    textfont=dict(color='white', size=12),
    customdata=df_top_topics[['topic_type', 'avg_score']].values,
    hovertemplate="<b>%{y}</b><br>Typ: %{customdata[0]}<br>Erwähnungen: %{x:,}<br>Score: %{customdata[1]:.2f}<extra></extra>"
))
fig_de.update_layout(
    title=dict(
        text=f"Top 15 Themen nach Erwähnungen<br><sub style='font-size:0.85em;'>{STAND_TEXT_DE}</sub>",
        x=0.5, xanchor='center', font=dict(size=22)
    ),
    xaxis_title='Anzahl Erwähnungen',
    yaxis_title='',
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#1a1a1a',
    font=dict(color='white', size=14),
    height=600,
    xaxis=dict(gridcolor='#333333'),
    yaxis=dict(gridcolor='#333333', tickfont=dict(size=14)),
    margin=dict(t=100, b=60, l=180, r=120)
)
save_fig(fig_de, "top_topics_by_mentions_de")
fig_de.show()

# English version
fig_en = go.Figure(go.Bar(
    x=df_top_topics['mentions'],
    y=df_top_topics['topic'],
    orientation='h',
    marker=dict(color=df_top_topics['avg_score'], colorscale='RdYlGn', cmin=-1, cmax=1, colorbar=dict(title='Score')),
    text=[f"{m:,} ({s:.2f})" for m, s in zip(df_top_topics['mentions'], df_top_topics['avg_score'])],
    textposition='outside',
    textfont=dict(color='white', size=12),
    customdata=df_top_topics[['topic_type', 'avg_score']].values,
    hovertemplate="<b>%{y}</b><br>Type: %{customdata[0]}<br>Mentions: %{x:,}<br>Score: %{customdata[1]:.2f}<extra></extra>"
))
fig_en.update_layout(
    title=dict(
        text=f"Top 15 Topics by Mentions<br><sub style='font-size:0.85em;'>{STAND_TEXT_EN}</sub>",
        x=0.5, xanchor='center', font=dict(size=22)
    ),
    xaxis_title='Number of Mentions',
    yaxis_title='',
    plot_bgcolor='#1a1a1a',
    paper_bgcolor='#1a1a1a',
    font=dict(color='white', size=14),
    height=600,
    xaxis=dict(gridcolor='#333333'),
    yaxis=dict(gridcolor='#333333', tickfont=dict(size=14)),
    margin=dict(t=100, b=60, l=180, r=120)
)
save_fig(fig_en, "top_topics_by_mentions_en")
fig_en.show()

## 8. Example Tweets with Sentiment

In [ ]:
def show_sentiment_examples(df, n=5, sentiment_filter=None):
    """Show example tweets with sentiment analysis."""
    filtered = df.copy()
    if sentiment_filter:
        filtered = filtered[filtered['sentiment'] == sentiment_filter]
    
    unique_tweets = filtered.drop_duplicates(subset=['tweet_id']).head(n)
    
    for _, row in unique_tweets.iterrows():
        tweet_sentiments = df[df['tweet_id'] == row['tweet_id']]
        
        print(f"\n{'='*80}")
        print(f"@{row['username']}:")
        print(f"{row['tweet_text'][:200]}..." if len(str(row['tweet_text'])) > 200 else row['tweet_text'])
        print(f"\nSentiment Analysis:")
        
        for _, s in tweet_sentiments.iterrows():
            emoji = "🟢" if s['sentiment'] == 'positive' else "🔴" if s['sentiment'] == 'negative' else "🟡"
            print(f"  {emoji} {s['topic']} ({s['topic_type']}): {s['sentiment']} (Score: {s['score']:.2f})")
            print(f"     → {s['reasoning']}")

print("\n=== POSITIVE EXAMPLES ===")
show_sentiment_examples(df_sentiment, n=3, sentiment_filter='positive')

print("\n\n=== NEGATIVE EXAMPLES ===")
show_sentiment_examples(df_sentiment, n=3, sentiment_filter='negative')

## 9. Run New Sentiment Analysis

Analyze tweets with default topics or custom topics (from trends, hashtags, etc.)

In [ ]:
from xminer.analysis.sentiment import (
    analyze_tweet_sentiment,
    save_sentiment_results,
    get_tweets_for_analysis,
    DEFAULT_POLITICAL_PARTIES,
    DEFAULT_POLICY_AREAS,
    DEFAULT_POLITICIANS
)

# Show available default topics
print("Default Political Parties:", DEFAULT_POLITICAL_PARTIES)
print("Default Policy Areas:", DEFAULT_POLICY_AREAS)
print("Default Politicians:", DEFAULT_POLITICIANS)

In [ ]:
# Example: Analyze with default topics (auto-detect parties, policies, politicians)
tweets_to_analyze = get_tweets_for_analysis(limit=5, exclude_analyzed=True)
print(f"Tweets to analyze: {len(tweets_to_analyze)}")

for tweet in tweets_to_analyze:
    print(f"\nAnalyzing @{tweet['username']}: {tweet['text'][:50]}...")
    
    results = analyze_tweet_sentiment(tweet['text'], topic_type='auto')
    
    if results:
        saved = save_sentiment_results(tweet['tweet_id'], results, tweet['author_id'])
        print(f"  ✓ Found {len(results)} topics, saved {saved} records")
    else:
        print(f"  - No relevant topics found")

In [ ]:
# Example: Analyze with CUSTOM topics (e.g., from trends or user input)
# Customize these topics based on current trends, hashtags, or specific interests

CUSTOM_TOPICS = [
    "#BTW25",           # Election hashtag
    "Koalition",        # Coalition talks
    "Neuwahlen",        # New elections
    "Ampel",            # Traffic light coalition
    "Schuldenbremse",   # Debt brake
]

# Get some tweets to analyze
tweets_to_analyze = get_tweets_for_analysis(limit=3, exclude_analyzed=False)

for tweet in tweets_to_analyze:
    print(f"\nAnalyzing @{tweet['username']}: {tweet['text'][:50]}...")
    
    # Use custom topics with custom_topic_type='trend'
    results = analyze_tweet_sentiment(
        tweet['text'], 
        topics=CUSTOM_TOPICS, 
        topic_type='custom',
        custom_topic_type='trend'  # This will label results as 'trend' type
    )
    
    if results:
        for r in results:
            print(f"  → {r.topic} ({r.topic_type}): {r.sentiment} (score: {r.score:.2f})")
            print(f"    {r.reasoning}")
    else:
        print(f"  - No relevant topics found")

## 10. Summary

In [ ]:
import os

print(f"\n{'='*80}")
print(f"Sentiment Analysis Complete!")
print(f"{'='*80}\n")

print(f"Total sentiment records: {len(df_sentiment):,}")
print(f"Unique tweets analyzed: {df_sentiment['tweet_id'].nunique():,}")
print(f"Unique topics: {df_sentiment['topic'].nunique()}")

print(f"\n--- By Topic Type ---")
for topic_type in df_sentiment['topic_type'].unique():
    subset = df_sentiment[df_sentiment['topic_type'] == topic_type]
    print(f"  {topic_type}: {len(subset):,} records, avg score: {subset['score'].mean():.2f}")

print(f"\n--- By Sentiment ---")
for sentiment in ['positive', 'negative', 'neutral', 'mixed']:
    count = len(df_sentiment[df_sentiment['sentiment'] == sentiment])
    pct = count / len(df_sentiment) * 100 if len(df_sentiment) > 0 else 0
    print(f"  {sentiment}: {count:,} ({pct:.1f}%)")

print(f"\n--- Charts saved to: {GRAPHICS_DIR} ---")
png_files = sorted(GRAPHICS_DIR.glob("*.png"))
for png_file in png_files:
    size = os.path.getsize(png_file) / 1024
    print(f"  📊 {png_file.name} ({size:.1f} KB)")

print(f"\n✅ {len(png_files)} chart(s) ready for social media posting!")